In [ ]:
# this is a parameter that will get overwritten when run by papermill on a schedules
is_local_development = True

In [ ]:
!python -m pip install gitlabdata --upgrade

In [ ]:
import configparser

# import sys
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import json, os
from pyprojroot import here
from os import environ as env

In [ ]:
from gitlabdata.orchestration_utils import (
    data_science_engine_factory,
    query_dataframe,
    snowflake_engine_factory,
    snowflake_stage_load_copy_remove,
    get_env_from_profile,
    dataframe_uploader,
    write_to_gsheets,
    read_from_gsheets,
    query_executor,
    query_from_file,
)

## Create Snowflake engine

In [ ]:
# engine factory can be created using a local role from output
# depending on this notebook being run locally or remotely, the
# engine is creation process is different

if is_local_development:
    snowflake_engine = data_science_engine_factory(
        profile_target="sales_analytics_local"
    )
else:
    snowflake_engine = snowflake_engine_factory(env, "SALES_ANALYTICS")

    raw_db_name = env["SNOWFLAKE_LOAD_DATABASE"]
    prod_db_name = env["SNOWFLAKE_PROD_DATABASE"]

snowflake_engine

## Credentials for Gsheet manipulation

Remember to give access to the following two users:

- Data Team runner: data-team-sheets-sa@gitlab-analysis.iam.gserviceaccount.com
- Sales Strategy service account: service-revenue-strat-analytic@revenue-strategy-anal-411d5a72.iam.gserviceaccount.com

In [ ]:
# read the credentials of the google service account
if is_local_development:
    credentials_path = here("credentials/rsa_gcloud_service_account.json")
    # credentials_path = here("credentials/gsheet_service_file.json")

    with open(credentials_path) as f:
        service_account_credentials = f.read().replace("\n", "")

    # set the credential as a enviroment variable
    os.environ["GSHEETS_SERVICE_ACCOUNT_CREDENTIALS"] = service_account_credentials

# X-Ray daily ETL update process

This notebook is run daily to update the X-Ray dashboard. 

The process is responsible of updating the following datasets:

1. QTD Metrics
2. Historical values subset

## Excute Snowflake query

In [ ]:
qtd_metrics = query_from_file(snowflake_engine, "xr_qtd_metrics_for_xray.sql")

In [ ]:
# Test
qtd_metrics.qtd_booked_net_arr.sum()

In [ ]:
hist_qtr_metrics = query_from_file(
    snowflake_engine, "xr_hist_qtr_coverages_for_xray.sql"
)
hist_qtr_metrics["last_extracted_etl"] = date.today()

In [ ]:
hist_qtr_metrics.head()

In [ ]:
hist_fy_n4q_metrics = query_from_file(
    snowflake_engine, "xr_hist_fy_n4q_coverages_for_xray.sql"
)

hist_fy_n4q_metrics["last_extracted_etl"] = date.today()

In [ ]:
hist_fy_n4q_metrics.head()

## QTD Metrics Extracts

### QTD Transform data into a pre-aggregated table

In [ ]:
# SQL output is imported as a dataframe variable called 'df'

df = qtd_metrics.copy()


# check the subset of columns and if see an na or an other
# set all the keys to other
def check_for_others_or_na(x):
    if ("other" in "\t".join(list(x))) or ("na" in list(x)):
        x = x.apply(lambda x: "other")

    return x


### considered keys
agg_key_list = [
    "key_overall",
    "key_sqs",
    "key_ot",
    "key_bu",
    "key_bu_ot",
    "key_bu_sqs",
    "key_bu_subbu",
    "key_bu_subbu_ot",
    "key_bu_subbu_sqs",
    "key_bu_subbu_division",
    "key_bu_subbu_division_ot",
    "key_bu_subbu_division_sqs",
    "key_bu_subbu_division_asm",
]

metrics_list = [
    "qtd_open_1plus_deal_count",
    "qtd_open_3plus_deal_count",
    "qtd_open_4plus_deal_count",
    "qtd_closed_deal_count",
    "qtd_pipe_gen_deal_count",
    "qtd_booked_net_arr",
    "qtd_open_1plus_net_arr",
    "qtd_open_3plus_net_arr",
    "qtd_open_4plus_net_arr",
    "qtd_pipe_gen_net_arr",
    "qtd_created_and_closed_net_arr",
    "rq_plus_1_open_1plus_deal_count",
    "rq_plus_1_open_3plus_deal_count",
    "rq_plus_1_open_4plus_deal_count",
    "rq_plus_1_open_1plus_net_arr",
    "rq_plus_1_open_3plus_net_arr",
    "rq_plus_1_open_4plus_net_arr",
    "rq_plus_2_open_1plus_deal_count",
    "rq_plus_2_open_3plus_deal_count",
    "rq_plus_2_open_4plus_deal_count",
    "rq_plus_2_open_1plus_net_arr",
    "rq_plus_2_open_3plus_net_arr",
    "rq_plus_2_open_4plus_net_arr",
    "minus_1_year_pipe_gen_net_arr",
    "cfy_open_1plus_deal_count",
    "cfy_open_3plus_deal_count",
    "cfy_open_4plus_deal_count",
    "cfy_closed_deal_count",
    "cfy_booked_net_arr",
    "cfy_open_1plus_net_arr",
    "cfy_open_1plus_under_1m_net_arr",
    "cfy_open_3plus_net_arr",
    "cfy_open_4plus_net_arr",
    "next_4q_open_1plus_deal_count",
    "next_4q_open_3plus_deal_count",
    "next_4q_open_4plus_deal_count",
    "next_4q_open_1plus_net_arr",
    "next_4q_open_3plus_net_arr",
    "next_4q_open_4plus_net_arr",
    "next_4q_booked_net_arr",
    "minus_1_year_total_pipe_gen_net_arr",
    "minus_1_year_pipe_gen_pacing",
]

# initialize the dataframe
results = None
##### Initialize a few artificial metrics and update the data types
df[metrics_list] = df[metrics_list].astype(float)

# create an artificial global key to calculate a global curve for the whole company
# df['KEY_OVERALL'] = 'global'
df["key_overall"] = "other"
df.loc[df["key_bu"] != "other", "key_overall"] = "global"

for agg_key_value in agg_key_list:
    temp_agg = df.groupby(agg_key_value)[metrics_list].sum().reset_index()

    temp_agg.rename({agg_key_value: "agg_key_value"}, inplace=True, axis=1)
    temp_agg["agg_key_name"] = agg_key_value

    if results is None:
        results = temp_agg.copy()
    else:
        results = results.append(temp_agg)

results.columns = results.columns.str.lower()

fields_order = [
    "agg_key_value",
    "agg_key_name",
    "qtd_booked_net_arr",
    "qtd_open_1plus_net_arr",
    "qtd_open_3plus_net_arr",
    "qtd_open_4plus_net_arr",
    "qtd_pipe_gen_net_arr",
    "rq_plus_1_open_1plus_net_arr",
    "rq_plus_1_open_3plus_net_arr",
    "rq_plus_1_open_4plus_net_arr",
    "rq_plus_2_open_1plus_net_arr",
    "rq_plus_2_open_3plus_net_arr",
    "rq_plus_2_open_4plus_net_arr",
    "qtd_created_and_closed_net_arr",
    "minus_1_year_pipe_gen_net_arr",
    "qtd_open_1plus_deal_count",
    "qtd_open_3plus_deal_count",
    "qtd_open_4plus_deal_count",
    "qtd_closed_deal_count",
    "qtd_pipe_gen_deal_count",
    "rq_plus_1_open_1plus_deal_count",
    "rq_plus_1_open_3plus_deal_count",
    "rq_plus_1_open_4plus_deal_count",
    "rq_plus_2_open_1plus_deal_count",
    "rq_plus_2_open_3plus_deal_count",
    "rq_plus_2_open_4plus_deal_count",
    "current_fiscal_quarter_day_normalised",
    "last_updated",
    "cfy_open_1plus_deal_count",
    "cfy_open_3plus_deal_count",
    "cfy_open_4plus_deal_count",
    "cfy_closed_deal_count",
    "cfy_booked_net_arr",
    "cfy_open_1plus_net_arr",
    "cfy_open_1plus_under_1m_net_arr",
    "cfy_open_3plus_net_arr",
    "cfy_open_4plus_net_arr",
    "next_4q_open_1plus_deal_count",
    "next_4q_open_3plus_deal_count",
    "next_4q_open_4plus_deal_count",
    "next_4q_booked_net_arr",
    "next_4q_open_1plus_net_arr",
    "next_4q_open_3plus_net_arr",
    "next_4q_open_4plus_net_arr",
    "minus_1_year_total_pipe_gen_net_arr",
    "minus_1_year_pipe_gen_pacing",
    "last_extracted_etl",
]

# I need to keep track in the dashboard of the latest day the extract was taking out, this is shown to the business users and used in the
# historical report
results["current_fiscal_quarter_day_normalised"] = max(
    df["current_fiscal_quarter_day_normalised"]
)
results["last_updated"] = date.today()
results["last_extracted_etl"] = date.today()

qtd_pre_aggregated = results[fields_order].copy()

# Calculate the ratio for pipe gen for each aggregated level
qtd_pre_aggregated["minus_1_year_pipe_gen_pacing"] = (
    qtd_pre_aggregated["minus_1_year_pipe_gen_net_arr"]
    / qtd_pre_aggregated["minus_1_year_total_pipe_gen_net_arr"]
)
qtd_pre_aggregated["minus_1_year_pipe_gen_pacing"].fillna(0, inplace=True)

In [ ]:
# Test QTD total
qtd_pre_aggregated[qtd_pre_aggregated["agg_key_name"] == "key_bu"].groupby(
    "agg_key_value"
).agg({"qtd_booked_net_arr": "sum"})

### Write to X-Ray source gSheet file

In [ ]:
# Write to GSheets
sheet_id = "1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk"
sheet_name = "jupyter_qtd_aggregated"
write_to_gsheets(sheet_id, sheet_name, qtd_pre_aggregated)

# Historical QTR Coverages Data Extract

In [ ]:
# Write to GSheets
sheet_id = "1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk"
sheet_name = "jupyter_hist_qtr_aggregated"
write_to_gsheets(sheet_id, sheet_name, hist_qtr_metrics)

# Historical FY / N4Q Coverages Data Extract

In [ ]:
# Write to GSheets
sheet_id = "1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk"
sheet_name = "jupyter_hist_fy_n4q_aggregated"
write_to_gsheets(sheet_id, sheet_name, hist_fy_n4q_metrics)

## Clari Forecast

In [ ]:
qtd_forecast = query_from_file(snowflake_engine, "xr_forecast_update.sql")

In [ ]:
# Write to GSheets
sheet_id = "1uzeZLHv1n6LIAgKHfqANwKquQoJCTRKx4dXVpJXhKgk"
sheet_name = "jupyter_clari_forecast"
write_to_gsheets(sheet_id, sheet_name, qtd_forecast)